In [1]:
import pandas as pd
import numpy as np
from preprocess import prepare_df
import torch.nn as nn
import torch
import torch.nn.functional as F
from model import Model

In [2]:
DEVICE = torch.device("cuda:" + str(4))

In [3]:
data = pd.read_csv('../data/df_final.csv', index_col=0).drop(columns=['lat', 'lon', 'elv','date','c4','whc'])
good_sites = pd.read_csv("../data/df_20210507.csv", low_memory=False )['sitename'].unique()

In [4]:
df_sensor, df_meta, df_gpp = prepare_df(data,sites=good_sites)

In [5]:
sites_to_train = list(range(len(df_sensor)))
sites_to_train.remove(0)
sites_to_test = [0]

x_train = [df_sensor[i].values for i in sites_to_train]
conditional_train = [df_meta[i].values for i in sites_to_train]
y_train = [df_gpp[i].values.reshape(-1,1) for i in sites_to_train]

x_test = [df_sensor[i].values for i in sites_to_test]
conditional_test = [df_meta[i].values for i in sites_to_test]
y_test = [df_gpp[i].values.reshape(-1,1) for i in sites_to_test]

In [6]:
INPUT_FEATURES = len(df_sensor[0].columns) 
HIDDEN_DIM = 256
CONDITIONAL_FEATURES = len(df_meta[0].columns)

In [14]:
model = Model(INPUT_FEATURES, CONDITIONAL_FEATURES, HIDDEN_DIM, False).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters())

In [17]:
import time
from sklearn.metrics import r2_score


for epoch in range(100):
    train_loss = 0.0
    train_r2 = 0.0
    start = time.time()
    model.train()
    for (x, y, conditional) in zip(x_train, y_train, conditional_train):
        x = torch.FloatTensor(x).to(DEVICE)
        y = torch.FloatTensor(y).to(DEVICE)
        c = torch.FloatTensor(conditional).to(DEVICE)

        if x.shape[0] == 0:
            continue
        
        y_pred = model(x, c)

        optimizer.zero_grad()
        loss = F.mse_loss( y_pred, y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        train_r2 += r2_score(y_true=y.detach().cpu().numpy(), y_pred=y_pred.detach().cpu().numpy())
    
    model.eval()
    with torch.no_grad():
          for (x, y, conditional) in zip(x_test, y_test, conditional_test):
            x = torch.FloatTensor(x).to(DEVICE)
            y = torch.FloatTensor(y).to(DEVICE)
            c = torch.FloatTensor(conditional).to(DEVICE)
            

            y_pred = model(x, c)


            test_loss = F.mse_loss( y_pred, y)
            test_r2 = r2_score(y_true=y.detach().cpu().numpy(), y_pred=y_pred.detach().cpu().numpy())
    
    end = time.time()       
    print(f"Epoch: {epoch+1} ({end-start:.2f}s)")
    print(f"Train loss: {train_loss / len(sites_to_train):.4f} | R2: {train_r2 / len(sites_to_train):.4f}")
    print(f"Test loss: {test_loss:.4f} | R2: {test_r2:.4f}") 

Epoch: 1 (5.53s)
Train loss: 0.1072 | R2: 0.8187
Test loss: 0.2897 | R2: 0.7101
Epoch: 2 (5.59s)
Train loss: 0.1068 | R2: 0.8191
Test loss: 0.2839 | R2: 0.7159
Epoch: 3 (6.25s)
Train loss: 0.1056 | R2: 0.8203
Test loss: 0.2816 | R2: 0.7182
Epoch: 4 (5.62s)
Train loss: 0.1048 | R2: 0.8211
Test loss: 0.2811 | R2: 0.7187
Epoch: 5 (5.61s)
Train loss: 0.1045 | R2: 0.8214
Test loss: 0.2802 | R2: 0.7196
Epoch: 6 (5.61s)
Train loss: 0.1043 | R2: 0.8216
Test loss: 0.2738 | R2: 0.7260
Epoch: 7 (5.68s)
Train loss: 0.1034 | R2: 0.8225
Test loss: 0.2672 | R2: 0.7326
Epoch: 8 (5.52s)
Train loss: 0.1020 | R2: 0.8239
Test loss: 0.2635 | R2: 0.7364
Epoch: 9 (5.65s)
Train loss: 0.1005 | R2: 0.8254
Test loss: 0.2635 | R2: 0.7363
Epoch: 10 (5.50s)
Train loss: 0.1006 | R2: 0.8253
Test loss: 0.2691 | R2: 0.7307
Epoch: 11 (5.62s)
Train loss: 0.1000 | R2: 0.8258
Test loss: 0.2605 | R2: 0.7394
Epoch: 12 (5.98s)
Train loss: 0.0987 | R2: 0.8272
Test loss: 0.2568 | R2: 0.7431
Epoch: 13 (5.48s)
Train loss: 0.0994 

KeyboardInterrupt: 